# Opprette og skrive PySpark DataFrame til felles datalager

## Datasett, tabeller og filer
<font size=2>Datasett er en organisert samling av data (jf. definisjonen av <a href="https://statswiki.unece.org/display/GSIMclick/Data+Set" target="Data Set (GSIM)">'Data Set'</a> i GSIM). Eksempler på datasett kan være tidsserier, utvalgsdata, forløpsdata, transaksjonsdata og registre.
</font>

## Dataframe
<font size=2>I pyspark er <code>DataFrame</code> et dimensjonalt datasett organisert i navngitte kolonner. Det tilsvarer konseptuelt en tabell i en relasjonsdatabase eller en dataramme (data frame) i R eller Python. <code>DataFrame</code> kan konstrueres fra mange kilder, som f.eks. strukturerte datafiler eller databaser. <code>DataFrame</code> API er også tilgjengelig i Scala, Java, Python og R.
</font>

#### Importere biblioteker (kode)

In [ ]:
import pyspark.sql.functions as F

from pyspark.sql.types import *

## Konstruere en Dataframe i pyspark
De neste paragrafene etableres tre datasett (<code>pyspark.sql.Dataframe</code>) for testformål.

Koden for å lage en dataframe i pyspark:
<br><code>sqlContext.createDataFrame(<i>verdier</i>, <i>skjema</i>)</code>

Skjema er en <code>StructType</code> med et array av <code>StructField</code> som definerer kolonnene:
<br><code>skjema = StructType([StructField('<i>kolonne A</i>', <i>datatype</i>, <i>obligatorisk</i>)], [StructField('<i>kolonne B</i>', <i>datatype</i>, <i>obligatorisk</i>)], ...)></code>

Mens verdier er et array av rekker av data: 
<br><code>verdier = [('data i kolonne A', 'data i kolonnne B', ...), ('data i kolonne A', 'data i kolonnne B', ...), ...]</code>

In [ ]:
schema_land_areal = StructType([\
    StructField('Land',StringType(), False),\
    StructField('Areal', IntegerType(), False),\
    StructField('Årstall', IntegerType(), False),\
    StructField('Landkode', StringType(), False),\
    StructField('Kilde', StringType(), False)]\
    )

# Datasettet inneholder dubletter, men vi bryr oss ikke om det nå:
#
# ('Norge',385178,2018,'NO','https://www.fn.no/Statistikk/Areal')
# ('Norge',400000,2020,'NO','RKR: Avrundet til nærmeste 10 000')
# ('Kroatia',56590,2018,'HR','https://www.fn.no/Statistikk/Areal')
# ('Kroatia',56590,2018,'HR','https://www.fn.no/Statistikk/Areal')
# ('Holland',41540,2018,'NL','https://www.fn.no/Statistikk/Areal')
# ('Nederland',41540,2018,'NL','https://www.fn.no/Statistikk/Areal')

value_land_areal = [\
    ('Norge',385178,2018,'NO','https://www.fn.no/Statistikk/Areal'),\
    ('Sverige',447430,2018,'SE','https://www.fn.no/Statistikk/Areal'),\
    ('Finland',338450,2018,'FI','https://www.fn.no/Statistikk/Areal'),\
    ('Island',103000,2018,'IS','https://www.fn.no/Statistikk/Areal'),\
    ('Grønland',2166086,2018,'GL','https://snl.no/Gr%C3%B8nland'),\
    ('Færøyene',1396,2018,'FO','https://snl.no/F%C3%A6r%C3%B8yene'),\
    ('Kroatia',56590,2018,'HR','https://www.fn.no/Statistikk/Areal'),\
    ('Frankrike',549087,2018,'FR','https://www.fn.no/Statistikk/Areal'),\
    ('Belgia',30530,2018,'BE','https://www.fn.no/Statistikk/Areal'),\
    ('Norge',400000,2020,'NO','RKR: Avrundet til nærmeste 10 000'),\
    ('Holland',41540,2018,'NL','https://www.fn.no/Statistikk/Areal'),\
    ('Bulgaria',111000,2018,'BG','https://www.fn.no/Statistikk/Areal'),\
    ('Romania',238400,2018,'RO','https://www.fn.no/Statistikk/Areal'),\
    ('Slovenia',20675,2018,'SI','https://www.fn.no/Statistikk/Areal'),\
    ('Spania',505935,2018,'ES','https://www.fn.no/Statistikk/Areal'),\
    ('Sveits',41290,2018,'CH','https://www.fn.no/Statistikk/Areal'),\
    ('Østerrike',83879,2018,'AT','https://www.fn.no/Statistikk/Areal'),\
    ('Tyskland',357580,2018,'DE','https://www.fn.no/Statistikk/Areal'),\
    ('Storbritannia',243610,2018,'GB','https://www.fn.no/Statistikk/Areal'),\
    ('Tsjekkia',78870,2018,'CZ','https://www.fn.no/Statistikk/Areal'),\
    ('Slovakia',49030,2018,'SK','https://www.fn.no/Statistikk/Areal'),\
    ('San Marino',60,2018,'SM','https://www.fn.no/Statistikk/Areal'),\
    ('Portugal',92226,2018,'PT','https://www.fn.no/Statistikk/Areal'),\
    ('Polen',312680,2018,'PL','https://www.fn.no/Statistikk/Areal'),\
    ('Nederland',41540,2018,'NL','https://www.fn.no/Statistikk/Areal'),\
    ('Luxembourg',2590,2018,'LU','https://www.fn.no/Statistikk/Areal'),\
    ('Kroatia',56590,2018,'HR','https://www.fn.no/Statistikk/Areal'),\
    ('Italia',301340,2018,'IT','https://www.fn.no/Statistikk/Areal'),\
    ('Danmark',42920,2018,'DK','https://www.fn.no/Statistikk/Areal')]


df_land_areal = spark.createDataFrame(value_land_areal, schema_land_areal)

df_land_areal.show()

In [ ]:
schema_land_kode = StructType([\
    StructField('Landkode',StringType(), False),\
    StructField('Land', StringType(), False)]\
    )

#landkodene her hentet fra Landkoder utenrikshandel 2011 (ssb.no) 5/5/2020

value_land_kode = [\
('AD','Andorra'),\
('AE','De Forente Arabiske Emirater'),\
('AF','Afghanistan'),\
('AG','Antigua og Barbuda'),\
('AI','Anguilla'),\
('AL','Albania'),\
('AM','Armenia'),\
('AO','Angola'),\
('AR','Argentina'),\
('AS','Amerikansk Samoa'),\
('AT','Østerrike'),\
('AU','Australia'),\
('AW','Aruba'),\
('AX','Åland'),\
('AZ','Aserbajdsjan'),\
('BA','Bosnia-Hercegovina'),\
('BB','Barbados'),\
('BD','Bangladesh'),\
('BE','Belgia'),\
('BF','Burkina Faso'),\
('BG','Bulgaria'),\
('BH','Bahrain'),\
('BI','Burundi'),\
('BJ','Benin'),\
('BL','Saint Barthelemy'),\
('BM','Bermuda'),\
('BN','Brunei Darussalam'),\
('BO','Bolivia'),\
('BQ','Bonaire,  Saint Eustatius og Saba'),\
('BR','Brasil'),\
('BS','Bahamas'),\
('BT','Bhutan'),\
('BW','Botswana'),\
('BY','Hviterussland'),\
('BZ','Belize'),\
('CA','Canada'),\
('CC','Kokosøyene (Keelingøyene)'),\
('CD','Kongo'),\
('CF','Sentralafrikanske Republikk'),\
('CG','Kongo,  Brazzaville'),\
('CH','Sveits'),\
('CI','Elfenbenskysten'),\
('CK','Cookøyene'),\
('CL','Chile'),\
('CM','Kamerun'),\
('CN','Kina'),\
('CO','Colombia'),\
('CR','Costa Rica'),\
('CU','Cuba'),\
('CV','Kapp Verde'),\
('CW','Curacao'),\
('CX','Christmasøya'),\
('CY','Kypros'),\
('CZ','Tsjekkia'),\
('DE','Tyskland'),\
('DJ','Djibouti'),\
('DK','Danmark'),\
('DM','Dominica'),\
('DO','Den Dominikanske Republikk'),\
('DZ','Algerie'),\
('EC','Ecuador'),\
('EE','Estland'),\
('EG','Egypt'),\
('EH','Vest-Sahara'),\
('ER','Eritrea'),\
('ES','Spania'),\
('ET','Etiopia'),\
('FI','Finland'),\
('FJ','Fiji'),\
('FK','Falklandsøyene (Malvinas)'),\
('FM','Mikronesiaføderasjonen'),\
('FO','Færøyene'),\
('FR','Frankrike'),\
('GA','Gabon'),\
('GB','Storbritannia'),\
('GD','Grenada'),\
('GE','Georgia'),\
('GF','Fransk Guyana'),\
('GG','Guernsey'),\
('GH','Ghana'),\
('GI','Gibraltar'),\
('GL','Grønland'),\
('GM','Gambia'),\
('GN','Guinea'),\
('GP','Guadeloupe'),\
('GQ','Ekvatorial-Guinea'),\
('GR','Hellas'),\
('GS','Sør-Georgia/Søndre Sandwichøyene'),\
('GT','Guatemala'),\
('GU','Guam'),\
('GW','Guinea-Bissau'),\
('GY','Guyana'),\
('HK','Hong Kong'),\
('HM','Heard- og McDonaldøyene'),\
('HN','Honduras'),\
('HR','Kroatia'),\
('HT','Haiti'),\
('HU','Ungarn'),\
('ID','Indonesia'),\
('IE','Irland'),\
('IL','Israel'),\
('IM','Isle of Man'),\
('IN','India'),\
('IO','Det Britiske terr. i Indiahavet'),\
('IQ','Irak'),\
('IR','Iran'),\
('IS','Island'),\
('IT','Italia'),\
('JE','Jersey'),\
('JM','Jamaica'),\
('JO','Jordan'),\
('JP','Japan'),\
('KE','Kenya'),\
('KG','Kirgisistan'),\
('KH','Kambodsja'),\
('KI','Kiribati'),\
('KM','Komorene'),\
('KN','Saint Kitts og Nevis'),\
('KP','Nord-Korea'),\
('KR','Sør-Korea'),\
('KW','Kuwait'),\
('KY','Caymanøyene'),\
('KZ','Kasakhstan'),\
('LA','Laos'),\
('LB','Libanon'),\
('LC','Saint Lucia'),\
('LI','Liechtenstein'),\
('LK','Sri Lanka'),\
('LR','Liberia'),\
('LS','Lesotho'),\
('LT','Litauen'),\
('LU','Luxemburg'),\
('LV','Latvia'),\
('LY','Libya'),\
('MA','Marokko'),\
('MC','Monaco'),\
('MD','Moldova'),\
('ME','Montenegro'),\
('MF','Saint Martin FR'),\
('MG','Madagaskar'),\
('MH','Marshalløyene'),\
('MK','Makedonia'),\
('ML','Mali'),\
('MM','Myanmar/Burma'),\
('MN','Mongolia'),\
('MO','Macao'),\
('MP','Nordre Marianene'),\
('MQ','Martinique'),\
('MR','Mauritania'),\
('MS','Montserrat'),\
('MT','Malta'),\
('MU','Mauritius'),\
('MV','Maldivene'),\
('MW','Malawi'),\
('MX','Mexico'),\
('MY','Malaysia'),\
('MZ','Mosambik'),\
('NA','Namibia'),\
('NC','Ny Caledonia'),\
('NE','Niger'),\
('NF','Norfolkøya'),\
('NG','Nigeria'),\
('NI','Nicaragua'),\
('NL','Nederland'),\
('NO','Norge'),\
('NP','Nepal'),\
('NR','Nauru'),\
('NT','Nøytralsone'),\
('NU','Niue'),\
('NZ','New Zealand'),\
('OM','Oman'),\
('PA','Panama'),\
('PE','Peru'),\
('PF','Fransk Polynesia'),\
('PG','Papua Ny-Guinea'),\
('PH','Filippinene'),\
('PK','Pakistan'),\
('PL','Polen'),\
('PM','Saint Pierre og Miquelon'),\
('PN','Pitcairn'),\
('PR','Puerto Rico'),\
('PS','Vestbredden/Gazastripen'),\
('PT','Portugal'),\
('PW','Palau'),\
('PY','Paraguay'),\
('QA','Qatar'),\
('RE','Reunion'),\
('RO','Romania'),\
('RS','Serbia'),\
('RU','Russland'),\
('RW','Rwanda'),\
('SA','Saudi-Arabia'),\
('SB','Salomonøyene'),\
('SC','Seychellene'),\
('SD','Sudan'),\
('SE','Sverige'),\
('SG','Singapore'),\
('SH','Sankt Helena'),\
('SI','Slovenia'),\
('SK','Slovakia'),\
('SL','Sierra Leone'),\
('SM','San Marino'),\
('SN','Senegal'),\
('SO','Somalia'),\
('SR','Surinam'),\
('ST','Sao Tome og Principe'),\
('SV','El Salvador'),\
('SX','Sint Marteen (Nederlandsk del)'),\
('SY','Syria'),\
('SZ','Swaziland'),\
('TC','Turks og Caicosøyene'),\
('TD','Tsjad'),\
('TF','Franske Sørlige Territorier'),\
('TG','Togo'),\
('TH','Thailand'),\
('TJ','Tadsjikistan'),\
('TK','Tokelau'),\
('TL','Øst-Timor'),\
('TM','Turkmenistan'),\
('TN','Tunisia'),\
('TO','Tonga'),\
('TR','Tyrkia'),\
('TT','Trinidad og Tobago'),\
('TV','Tuvalu'),\
('TW','Taiwan'),\
('TZ','Tanzania'),\
('UA','Ukraina'),\
('UG','Uganda'),\
('UM','USA mindre utenforliggende øyer'),\
('US','USA'),\
('UY','Uruguay'),\
('UZ','Usbekistan'),\
('VA','Vatikanstaten'),\
('VC','Saint Vincent og Grenadine'),\
('VE','Venezuela'),\
('VG','Jomfruøyene Britisk'),\
('VI','Jomfruøyene US'),\
('VN','Vietnam'),\
('VU','Vanuatu'),\
('WF','Wallis og Futuna'),\
('WS','Samoa'),\
('XB','Kanariøyene'),\
('XC','Ceuta og Melilla'),\
('XK','Kosovo'),\
('YE','Jemen'),\
('YT','Mayotte'),\
('ZA','Sør-Afrika'),\
('ZM','Zambia'),\
('ZW','Zimbabwe'),\
('ZZ','Internasjonale Organisasjoner')]

df_land_kode = spark.createDataFrame(value_land_kode, schema_land_kode)

df_land_kode.show()

In [ ]:
schema_land_innbyggerantall = StructType([\
    StructField('Land',StringType(), False),\
    StructField('Landkode',StringType(), False),\
    StructField('Innbyggerantall', IntegerType(), False),\
    StructField('År', IntegerType(), False),\
    StructField('Kilde', StringType(), False)])

# Datasettet inneholder dubletter, men vi bryr oss ikke om det nå:
# 
# ('Norge','NO',5449693,2020,'https://www.fn.no/Statistikk/Befolkningstall')
# ('Norge','NO',5353363,2018,'https://www.fn.no/Statistikk/Befolkningstall')
# ('Norge','NO',5224323,2020,'https://www.fn.no/Statistikk/Befolkningstall') (FEIL)
# ('Frankrike','FR',65721165,2020,'https://www.fn.no/Statistikk/Befolkningstall')
# ('Frankrike','FR',65721165,2020,'https://www.fn.no/Statistikk/Befolkningstall')
# ('Frankrike','FR',65721165,2020,'https://www.fn.no/Statistikk/Befolkningstall')

 
value_land_innbyggerantall_2020 = [\
    ('Norge','NO',5449693,2020,'https://www.fn.no/Statistikk/Befolkningstall'),\
    ('Sverige','SE',10121686,2020,'https://www.fn.no/Statistikk/Befolkningstall'),\
    ('Finland','FI',5580127,2020,'https://www.fn.no/Statistikk/Befolkningstall'),\
    ('Island','IS',343228,2020,'https://www.fn.no/Statistikk/Befolkningstall'),\
    ('Færøyene','FO',0,2020,'https://www.fn.no/Statistikk/Befolkningstall'),\
    ('Frankrike','FR',65721165,2020,'https://www.fn.no/Statistikk/Befolkningstall'),\
    ('Norge','NO',5353363,2018,'https://www.fn.no/Statistikk/Befolkningstall'),\
    ('Frankrike','FR',65721165,2020,'https://www.fn.no/Statistikk/Befolkningstall'),\
    ('Belgia','BE',11619972,2020,'https://www.fn.no/Statistikk/Befolkningstall'),\
    ('Bulgaria','BG',6940527,2020,'https://www.fn.no/Statistikk/Befolkningstall'),\
    ('Slovenia','SI',2082055,2020,'https://www.fn.no/Statistikk/Befolkningstall'),\
    ('Spania','ES',46459219,2020,'https://www.fn.no/Statistikk/Befolkningstall'),\
    ('Sveits','CH',8670535,2020,'https://www.fn.no/Statistikk/Befolkningstall'),\
    ('Østerrike','AT',8782210,2020,'https://www.fn.no/Statistikk/Befolkningstall'),\
    ('Tyskland','DE',82540450,2020,'https://www.fn.no/Statistikk/Befolkningstall'),\
    ('Storbritannia','GB',67334208,2020,'https://www.fn.no/Statistikk/Befolkningstall'),\
    ('Frankrike','FR',65721165,2020,'https://www.fn.no/Statistikk/Befolkningstall'),\
    ('San Marino','SM',33809,2020,'https://www.fn.no/Statistikk/Befolkningstall'),\
    ('Portugal','PT',10218413,2020,'https://www.fn.no/Statistikk/Befolkningstall'),\
    ('Polen','PL',37942231,2020,'https://www.fn.no/Statistikk/Befolkningstall'),\
    ('Nederland','NL',17181248,2020,'https://www.fn.no/Statistikk/Befolkningstall'),\
    ('Luxembourg','LU',603944,2020,'https://www.fn.no/Statistikk/Befolkningstall'),\
    ('Kroatia','HR',4115947,2020,'https://www.fn.no/Statistikk/Befolkningstall'),\
    ('Norge','NO',5224323,2020,'https://www.fn.no/Statistikk/Befolkningstall'),\
    ('Italia','IT',59132073,2020,'https://www.fn.no/Statistikk/Befolkningstall'),\
    ('Danmark','DK',5796800,2020,'https://www.fn.no/Statistikk/Befolkningstall')]

df_land_innbyggerantall_2020 = spark.createDataFrame(value_land_innbyggerantall_2020, schema_land_innbyggerantall)

value_land_innbyggerantall_2018 = [\
    ('Færøyene','FO',48497,2018,'https://data.worldbank.org/country/faroe-islands'),\
    ('Grønland','GL',56025,2018,'https://data.worldbank.org/country/greenland'),\
    ('Caymanøyene','KY',64174,2018,'https://data.worldbank.org/country/cayman-islands'),\
    ('Bermuda','BM',63973,2018,'https://data.worldbank.org/country/bermuda')]

df_land_innbyggerantall_2018 = spark.createDataFrame(value_land_innbyggerantall_2018, schema_land_innbyggerantall)


schema_land_innbyggerantall_2017 = StructType([\
    StructField('Landkode',StringType(), False),\
    StructField('Land',StringType(), False),\
    StructField('Innbyggerantall', IntegerType(), False),\
    StructField('År', StringType(), False),\
    StructField('Kilde', StringType(), False)])
    
value_land_innbyggerantall_2017 = [\
    ('FO','Færøyene',47497,2017,'https://data.worldbank.org/country/faroe-islands'),\
    ('GL','Grønland',55025,2017,'https://data.worldbank.org/country/greenland'),\
    ('KY','Caymanøyene',63123,2017,'https://data.worldbank.org/country/cayman-islands'),\
    ('BM','Bermuda',62129,2017,'https://data.worldbank.org/country/bermuda')]   
    
df_land_innbyggerantall_2017 = spark.createDataFrame(value_land_innbyggerantall_2017, schema_land_innbyggerantall_2017)  

df_land_innbyggerantall_2020.show()
df_land_innbyggerantall_2018.show()
df_land_innbyggerantall_2017.show()

In [ ]:
schema_land_bnp = StructType([\
    StructField('Land',StringType(), False),\
    StructField('Landkode',StringType(), False),\
    StructField('BNP', LongType(), False),\
    StructField('År', IntegerType(), False),\
    StructField('Kilde', StringType(), False)])

# ('Slovenia','SI',-54007972106,2018,'https://www.fn.no/Statistikk/BNP') (feil: negativ verdi)
# ('Portugal','PT',-25878475760,2018,'https://www.fn.no/Statistikk/BNP') (feil: negativ verdi)
 
value_land_bnp = [\
    ('Norge','NO',434166615432,2018,'https://www.fn.no/Statistikk/BNP'),\
    ('Sverige','SE',556086488937,2018,'https://www.fn.no/Statistikk/BNP'),\
    ('Finland','FI',355675329086,2018,'https://www.fn.no/Statistikk/BNP'),\
    ('Island','IS',25878475760,2018,'https://www.fn.no/Statistikk/BNP'),\
    ('Frankrike','FR',2777535239277,2018,'https://www.fn.no/Statistikk/BNP'),\
    ('Belgia','BE',542761092103,2018,'https://www.fn.no/Statistikk/BNP'),\
    ('Bulgaria','BG',65132951116,2018,'https://www.fn.no/Statistikk/BNP'),\
    ('Slovenia','SI',-54007972106,2018,'https://www.fn.no/Statistikk/BNP'),\
    ('Spania','ES',1419041949910,2018,'https://www.fn.no/Statistikk/BNP'),\
    ('Sveits','CH',705140354166,2018,'https://www.fn.no/Statistikk/BNP'),\
    ('Østerrike','AT',455285818035,2018,'https://www.fn.no/Statistikk/BNP'),\
    ('Tyskland','DE',3947620162502,2018,'https://www.fn.no/Statistikk/BNP'),\
    ('Storbritannia','GB',2855296731521,2018,'https://www.fn.no/Statistikk/BNP'),\
    ('Romania','RO',239552516744,2018,'https://www.fn.no/Statistikk/BNP'),\
    ('Portugal','PT',-25878475760,2018,'https://www.fn.no/Statistikk/BNP'),\
    ('Polen','PL',585663814824,2018,'https://www.fn.no/Statistikk/BNP'),\
    ('Nederland','NL',913658465709,2018,'https://www.fn.no/Statistikk/BNP'),\
    ('Luxembourg','LU',70885325883,2018,'https://www.fn.no/Statistikk/BNP'),\
    ('Kroatia','HR',60971699315,2018,'https://www.fn.no/Statistikk/BNP'),\
    ('Italia','IT',2083864259623,2018,'https://www.fn.no/Statistikk/BNP'),\
    ('Danmark','DK',355675329086,2018,'https://www.fn.no/Statistikk/BNP')]

df_land_bnp = spark.createDataFrame(value_land_bnp, schema_land_bnp)

df_land_bnp.show()

In [ ]:
# Setter Sveriges BNP til missing
df_land_bnp = df_land_bnp.withColumn('BNP',F.when(df_land_bnp.Landkode == 'SE',F.lit(None).cast(NullType())).otherwise(df_land_bnp.BNP))
df_land_bnp.show()

## Hvordan lagre data til datalager

Koden for å skrive datasettene (DataFrame) til datalageret på dataplattformen:

<code> [DATAFRAME_NAME].write\\
    .format("gsim")\\
    .option("valuation", "[VALUATION_VALUE]")\\
    .option("state", "[STATE_VALUE]")\\
    .path("[STORE_PATH]")
</code>

Verdiene for <code>valuation</code> (verdivurdering) og <code>state</code> (datatilstander) er beskrevet i kode: https://github.com/statisticsnorway/dapla-catalog-protobuf/blob/master/src/main/proto/dataset.proto

Koden for å skrive datasettene laget i dette notatet følger i de neste paragrafene

### Skrive datasettet df_land_kode til datalageret (kode)

In [ ]:
df_land_kode.write\
    .format("gsim")\
    .option("valuation", "INTERNAL")\
    .option("state", "INPUT")\
    .path('/felles/veiledning/pyspark/eksempler/landkode')

### Skrive datasettet df_land_areal til datalageret (kode)

In [ ]:
df_land_areal.write\
    .format("gsim")\
    .option("valuation", "INTERNAL")\
    .option("state", "INPUT")\
    .path('/felles/veiledning/pyspark/eksempler/areal')

### Skrive datasettet df_land_bnp til datalageret (kode)

In [ ]:
df_land_bnp.write\
    .format("gsim")\
    .option("valuation", "INTERNAL")\
    .option("state", "INPUT")\
    .path('/felles/veiledning/pyspark/eksempler/bnp')

### Skrive datasettet df_land_innbyggerantall_2020 til datalageret (kode)

In [ ]:
df_land_innbyggerantall_2020.write\
    .format("gsim")\
    .option("valuation", "INTERNAL")\
    .option("state", "INPUT")\
    .path('/felles/veiledning/pyspark/eksempler/innbyggerantall/2020')

### Skrive datasettet df_land_innbyggerantall_2018 til datalageret (kode)

In [ ]:
df_land_innbyggerantall_2018.write\
    .format("gsim")\
    .option("valuation", "INTERNAL")\
    .option("state", "INPUT")\
    .path('/felles/veiledning/pyspark/eksempler/innbyggerantall/2018')

### Skrive datasettet df_land_innbyggerantall_2017 til datalageret (kode)

In [ ]:
df_land_innbyggerantall_2017.write\
    .format("gsim")\
    .option("valuation", "INTERNAL")\
    .option("state", "INPUT")\
    .path('/felles/veiledning/pyspark/eksempler/innbyggerantall/2017')